# Digit Recognititon on new dataset using CNN (Part 3 of Task 2)

Objective:

>Take the following [dataset](https://www.dropbox.com/s/otc12z2w7f7xm8z/mnistTask3.zip), train on this dataset and
provide test accuracy on the MNIST test set, using the same test split from part 2. Train
using scratch random initialization and using the pretrained network part 1. Do the same
analysis as 2 and report what happens this time. Try and do qualitative analysis of what's
different in this dataset. Please save your model checkpoints.

In [1]:
import random
import copy
import time 
import pandas as pd 
from PIL import Image

import torch
import torchvision
import torch.optim as optim
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [2]:
from data import dataset
from plotting import plot_loss, plot_images
from model import Net
from train import train_model, compute_accuracy, cal_accuracy

To change images into vector with normalisation and augmentation, following values are fetched to torchvision.transforms function. We perform: 
    
    - Random Rotations
    - Random Resizing and Cropping 
    - Change Image arrays to Tensor
    - Normalize

In [3]:
train_transforms = transforms.Compose([
                   transforms.RandomRotation(degrees=45),
                   transforms.RandomResizedCrop(64, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
                   transforms.ToTensor(),
                   transforms.Normalize(mean = [0.5,0.5,0.5], std = [0.5,0.5,0.5])

               ])

In [ ]:
data_folder = '../input/task3/mnistTask'
new_data = datasets.ImageFolder(root = data_folder, transform=train_transforms)

In [ ]:
VALID_RATIO = 0.83
new_train_data = int(len(new_data) * VALID_RATIO)
new_valid_data = len(new_data) - new_train_data
train, valid = data.random_split(new_data, [new_train_data, new_valid_data])

valid = copy.deepcopy(valid)
valid.dataset.transform = train_transforms

print(f'Number of training examples: {len(train)}')
print(f'Number of validation examples: {len(valid)}')

In [ ]:
train_loader = data.DataLoader(train, batch_size = 1024)
valid_loader = data.DataLoader(valid, batch_size = 1024)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model_scratch = Net(10)
model_scratch.to(device)

criterion_scratch = torch.nn.CrossEntropyLoss()
optimizer_scratch = torch.optim.Adam(model_scratch.parameters(), lr=0.1)
device

In [ ]:
model_scratch

In [ ]:
train_loss_scratch, valid_loss_scratch, train_acc_scratch, valid_acc_scratch = train_model(model=model_scratch, #Scratch model 
                                                                                 num_epochs=20,
                                                                                 train_iterator=train_loader,
                                                                                 valid_iterator=valid_loader, 
                                                                                 optimizer=optimizer_scratch,
                                                                                 criterion=criterion_scratch, 
                                                                                device=device, 
                                                                                model_save = False,
                                                                                model_name="model_scratch.pt")

In [ ]:
plot_loss(train_loss_scratch, valid_loss_scratch)

In [ ]:
plot_loss(train_acc_scratch, valid_acc_scratch)

## Pretrained Model

In [ ]:
model_trained = Net(62)
model_trained.load_state_dict(torch.load('model.pt'))
model_trained.train()

In [ ]:
model_trained.classifier[4] = torch.nn.Linear(4096,1024)
model_trained.classifier[6] = torch.nn.Linear(1024,10)

In [ ]:
model_trained.to(device)
criterion_num = torch.nn.CrossEntropyLoss()
optimizer_num = torch.optim.Adam(model_trained.parameters(), lr=1e-4)
device

In [ ]:
model_trained

In [ ]:
train_loss_pretrained, valid_loss_pretrained, train_acc_pretrained, valid_acc_pretrained = train_model(model=model_trained, 
                                                                 #Pretrained model
                                                                 num_epochs=20,
                                                                 train_iterator=train_loader,
                                                                 valid_iterator=valid_loader, 
                                                                 optimizer=optimizer_trained_new,
                                                                 criterion=criterion_trained_new, 
                                                                device=device, model_name="model_pretr.pt")

In [ ]:
plot_loss(train_loss_pretrained, train_loss_pretrained)

In [ ]:
plot_loss(valid_loss_pretrained, train_loss_pretrained)

In [ ]:
cal_accuracy(train_loader, model_trained, device)

In [ ]:
cal_accuracy(valid_loader, model_trained, device)

## Conclusion

# Thank you!

- Author : Pratik Kumar
- Date: April 2021
- Submitted to: MIDAS @IIITD
- References: 

    - [AlexNet Paper](https://papers.nips.cc/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf)
    - [MNIST](http://yann.lecun.com/exdb/mnist/)
    - [PyTorch Tutorials](https://pytorch.org/tutorials/)